In [8]:
from ttictoc import Timer
import pickle
import json
from ast import literal_eval

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score

import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
base = '/Users/chuamelia/Google Drive/Spring 2020/Machine Learning/fake-review-detection-project/data/processed/dev/'

def load_obj(fname,  base=base):
    # This loads the pickled object.
    with open(base + fname + '.pkl', 'rb') as f:
        return pickle.load(f)


def writeJsonFile(fname, data,  base=base):
    with open(base + fname +'.json', 'w') as outfile:
        json.dump(data, outfile)
    print('Successfully written to {}'.format(fname))
    
def readJsonFile(fname, base=base):
    with open(base + fname + '.json', 'r') as f:
        data = json.load(f)
    return data

In [10]:
def ClassifierMetrics (X_train, Y_train, X_test, Y_test, fitted_model):
    Y_pred = fitted_model.predict(X_test)
    Y_score = fitted_model.decision_function(X_test)
    metrics = {'train_accuracy': fitted_model.score(X_train, Y_train),
               'test_accuracy': fitted_model.score(X_test, Y_test),
               'test_auc_pred': roc_auc_score(Y_test, Y_pred),
               'test_auc_score': roc_auc_score(Y_test, Y_score),
               'test_ap_pred': average_precision_score(Y_test, Y_pred),
               'test_ap_score': average_precision_score(Y_test, Y_score)}
    return metrics

In [11]:
def fitted_lsvm(params, X_train, Y_train):
    svm = LinearSVC(**params)
    fitted_svm = svm.fit(X_train, Y_train)
    return fitted_svm


def fitted_lsvm_sgd(params, X_train, Y_train):
    svm = SGDClassifier(**params)
    fitted_svm = svm.fit(X_train, Y_train)
    return fitted_svm

In [12]:
def identity_tokenizer(tokens):
    return tokens

def tfidf_lsvm_sgd_metrics(params, dev_name, train_num):
    i = train_num
    train_fname = '../../data/processed/dev/ac4119_train_set_{0}_w_tokens.csv'.format(i)
    dev_fname = '../../data/processed/dev/{0}.csv'.format(dev_name)

    train = pd.read_csv(train_fname)
    dev = pd.read_csv(dev_fname)

    Y_train = train['label']
    Y_dev = dev['label']

    tfidf_vectorizer_fname = 'ac4119_X_train_set_{0}_tfidf_vectorizer'.format(i)
    tfidf_vectorizer = load_obj(tfidf_vectorizer_fname)

    train['token_review'] = train['token_review'].apply(lambda x: literal_eval(x))
    dev['token_review'] = dev['token_review'].apply(lambda x: literal_eval(x))

    X_train = tfidf_vectorizer.transform(train['token_review'])
    X_dev = tfidf_vectorizer.transform(dev['token_review'])

    fitted_model = fitted_lsvm_sgd(params, X_train ,Y_train )
    metrics = ClassifierMetrics(X_train, Y_train, X_dev, Y_dev, fitted_model)
    return metrics 

In [13]:
all_attempts = []

In [14]:
losses = ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron']
penalties = ['l2', 'l1', 'elasticnet']
alphas = [0.0001, 0.001, 0.01] # the higher the value, the stronger the regularization
# l1_ratio # elasticnet only 
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifierlsvm_params_combos = [(c,l,p) for c in cs for l in losses for p in penalty]
sgd_params_combos = [(a,l,p) for a in alphas for l in losses for p in penalties]

for train_num in range(8):
    other_train_sets = ['ac4119_train_set_{0}_w_tokens'.format(i) for i in range(8) if i != train_num]
    dev_sets = ['ac4119_dev_w_tokens'] + other_train_sets
    
    for dev_name in dev_sets: 
   
        params = {'random_state': 519,
                  'class_weight':'balanced'}

        t = Timer()
        t.start()

        metrics = tfidf_lsvm_sgd_metrics(params, dev_name, train_num)
        elapsed = t.stop()

        print('Elapsed time:',elapsed)

        model_attempt_details = {'elapsed_time': elapsed,
                                 'train_set':train_num,
                                 'dev_set': dev_name,
                                 'vectorizer':'tfidf',
                                 'model': 'SGDClassifier', 
                                 'params': params, 'metrics': metrics}

        all_attempts.append(model_attempt_details)

/Users/chuamelia/anaconda/envs/py35/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Elapsed time: 15.489575903862715


/Users/chuamelia/anaconda/envs/py35/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Elapsed time: 14.665767678059638


/Users/chuamelia/anaconda/envs/py35/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


...

Elapsed time: 12.96554045798257


/Users/chuamelia/anaconda/envs/py35/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Elapsed time: 8.66901442501694


/Users/chuamelia/anaconda/envs/py35/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Elapsed time: 14.06510992301628


In [ ]:
losses = ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron']
penalties = ['l2', 'l1', 'elasticnet']
alphas = [0.0001, 0.001, 0.01] # the higher the value, the stronger the regularization
# l1_ratio # elasticnet only 
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifierlsvm_params_combos = [(c,l,p) for c in cs for l in losses for p in penalty]
sgd_params_combos = [(a,l,p) for a in alphas for l in losses for p in penalties]

for train_num in range(8):
    other_train_sets = ['ac4119_train_set_{0}_w_tokens'.format(i) for i in range(8) if i != train_num]
    dev_sets = ['ac4119_dev_w_tokens'] + other_train_sets
    
    for dev_name in dev_sets: 
        for alpha, loss, penalty in sgd_params_combos:
            
            params = {'random_state': 519, 
                      'alpha': alpha, 
                      'loss': loss, 
                      'penalty': penalty, 
                      'class_weight':'balanced'}
            try:
                t = Timer()
                t.start()

                metrics = tfidf_lsvm_sgd_metrics(params, dev_name, train_num)
                elapsed = t.stop()

                print('Elapsed time:',elapsed)

                model_attempt_details = {'elapsed_time': elapsed,
                                         'train_set':train_num,
                                         'dev_set': dev_name,
                                         'vectorizer':'tfidf',
                                         'model': 'SGDClassifier', 
                                         'params': params, 'metrics': metrics}

                all_attempts.append(model_attempt_details)
            except:
                print('({0},{1},{2}) did not work.'.format(alpha, loss, penalty ))
                pass

/Users/chuamelia/anaconda/envs/py35/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Elapsed time: 16.667905900161713


/Users/chuamelia/anaconda/envs/py35/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Elapsed time: 16.091142923105508


/Users/chuamelia/anaconda/envs/py35/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


...

/Users/chuamelia/anaconda/envs/py35/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Elapsed time: 12.547408510930836


/Users/chuamelia/anaconda/envs/py35/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Elapsed time: 12.141005071345717


/Users/chuamelia/anaconda/envs/py35/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
# File name of the model attempts/results
fname = 'all_attempts_ac4119_20200516b'
writeJsonFile(fname, all_attempts)